In [1]:
import sqlite3
import utils
import pandas as pd
from sklearn.model_selection import train_test_split
from numba import cuda

from sklearn.metrics import f1_score
from sklearn.dummy import DummyClassifier

from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

import numpy as np
import torch

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

from keras.layers import Conv1D, GlobalMaxPooling1D,LSTM

C:\Users\X\AppData\Local\Continuum\anaconda3\envs\mlgpu\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [2]:
import keras.backend as K

#dtype='float16'
#K.set_floatx(dtype)

# default is 1e-7 which is too small for float16.  Without adjusting the epsilon, we will get NaN predictions because of divide by zero problems
#K.set_epsilon(1e-3) 

In [3]:
import tensorflow as tf

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10606027626498414059
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4557766656
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12167576636417620189
physical_device_desc: "device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [5]:
def free_gpu_mem():
    cuda.select_device(0)
    cuda.close()

In [6]:
def limit_mem():
    K.get_session().close()
    cfg = K.tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    K.set_session(K.tf.Session(config=cfg))

In [7]:
word_vec_size=100

In [8]:
def w2v_per_token_wiki(word_vec_size):
    conn=sqlite3.connect('songs.db')
    cur=conn.cursor()
    example_text=cur.execute("select Lyrics from song where Lyrics is not null").fetchall()
    labels=[i[0] for i in cur.execute("select Economic from song where Lyrics is not null").fetchall()]
    songs=[]
    for i in example_text:
        songs.append(utils.tokenize_word(utils.clean_text(i[0])))

    embeddings_index,songs=utils.get_embedding_dict(songs,'glove-wiki-gigaword-100')
    max_words=len(embeddings_index.keys())
    X,y=np.array(songs),np.array(labels)

    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(X)
    sequences = tokenizer.texts_to_sequences(X)

    length = []
    for x in X:
        length.append(len(x))

    X = pad_sequences(sequences, maxlen=max(length)+2)

    embedding_matrix = np.zeros((max_words, word_vec_size))
    for word, i in tokenizer.word_index.items():
        if i >= max_words:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.5)
    
    
    return x_train,x_test,y_train,y_test,embedding_matrix,max(length)+2,max_words

In [9]:
x_train,x_test,y_train,y_test,embedding_matrix,max_sent_len,max_words=w2v_per_token_wiki(word_vec_size)

In [10]:
def dummy(x_train,x_test,y_train,y_test):
    dummy = DummyClassifier()
    dummy.fit(x_train, y_train)
    y_pred=dummy.predict(x_test)
    return f1_score(y_test,y_pred)

In [11]:
print(dummy(x_train,x_test,y_train,y_test))

0.4360644497771683


In [14]:
def dnn_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model_ptw2v = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=False)
    model_ptw2v.add(e)
    model_ptw2v.add(Flatten())
    model_ptw2v.add(Dense(256, activation='relu'))
    model_ptw2v.add(Dropout(0.25))
    model_ptw2v.add(Dense(1, activation='sigmoid'))
    model_ptw2v.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model_ptw2v.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=256, verbose=2)
    y_pred = [int(round(i[0])) for i in model_ptw2v.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [15]:
print(dnn_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 3327 samples, validate on 3328 samples
Epoch 1/100
 - 4s - loss: 1.3249 - acc: 0.5428 - val_loss: 0.6357 - val_acc: 0.5856
Epoch 2/100
 - 1s - loss: 0.5629 - acc: 0.6844 - val_loss: 0.5964 - val_acc: 0.6433
Epoch 3/100
 - 1s - loss: 0.4659 - acc: 0.7761 - val_loss: 0.5823 - val_acc: 0.6496
Epoch 4/100
 - 1s - loss: 0.3611 - acc: 0.8605 - val_loss: 0.5899 - val_acc: 0.6436
Epoch 5/100
 - 1s - loss: 0.2777 - acc: 0.9170 - val_loss: 0.5991 - val_acc: 0.6481
Epoch 6/100
 - 1s - loss: 0.2050 - acc: 0.9441 - val_loss: 0.6417 - val_acc: 0.6457
Epoch 7/100
 - 1s - loss: 0.1538 - acc: 0.9651 - val_loss: 0.6710 - val_acc: 0.6535
Epoch 8/100
 - 1s - loss: 0.1149 - acc: 0.9766 - val_loss: 0.7147 - val_acc: 0.6596
Epoch 9/100
 - 1s - loss: 0.0980 - acc: 0.9793 - val_loss: 0.7507 - val_acc: 0.6556
Epoch 10/100
 - 1s - loss: 0.0789

 - 1s - loss: 0.0203 - acc: 0.9925 - val_loss: 1.5223 - val_acc: 0.6484
Epoch 90/100
 - 1s - loss: 0.0263 - acc: 0.9934 - val_loss: 1.5896 - val_acc: 0.6451
Epoch 91/100
 - 1s - loss: 0.0295 - acc: 0.9931 - val_loss: 1.3532 - val_acc: 0.6511
Epoch 92/100
 - 1s - loss: 0.0229 - acc: 0.9925 - val_loss: 1.4246 - val_acc: 0.6463
Epoch 93/100
 - 1s - loss: 0.0251 - acc: 0.9925 - val_loss: 1.4272 - val_acc: 0.6502
Epoch 94/100
 - 1s - loss: 0.0258 - acc: 0.9925 - val_loss: 1.4119 - val_acc: 0.6538
Epoch 95/100
 - 1s - loss: 0.0237 - acc: 0.9931 - val_loss: 1.4618 - val_acc: 0.6547
Epoch 96/100
 - 1s - loss: 0.0236 - acc: 0.9928 - val_loss: 1.4985 - val_acc: 0.6502
Epoch 97/100
 - 1s - loss: 0.0243 - acc: 0.9916 - val_loss: 1.5235 - val_acc: 0.6517
Epoch 98/100
 - 1s - loss: 0.0237 - acc: 0.9925 - val_loss: 1.5256 - val_acc: 0.6514
Epoch 99/100
 - 1s - loss: 0.0241 - acc: 0.9931 - val_loss: 1.5489 - val_acc: 0.6445
Epoch 100/100
 - 1s - loss: 0.0201 - acc: 0.9931 - val_loss: 1.5988 - val_acc:

In [16]:
def dnn_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model_ptw2v = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=True)
    model_ptw2v.add(e)
    model_ptw2v.add(Flatten())
    model_ptw2v.add(Dense(256, activation='relu'))
    model_ptw2v.add(Dropout(0.25))
    model_ptw2v.add(Dense(1, activation='sigmoid'))
    model_ptw2v.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model_ptw2v.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=256, verbose=2)
    y_pred = [int(round(i[0])) for i in model_ptw2v.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [17]:
print(dnn_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Train on 3327 samples, validate on 3328 samples
Epoch 1/100
 - 4s - loss: 1.3042 - acc: 0.5452 - val_loss: 0.6662 - val_acc: 0.5493
Epoch 2/100
 - 1s - loss: 0.6265 - acc: 0.6333 - val_loss: 0.6157 - val_acc: 0.6175
Epoch 3/100
 - 1s - loss: 0.5295 - acc: 0.7106 - val_loss: 0.5788 - val_acc: 0.6460
Epoch 4/100
 - 1s - loss: 0.4002 - acc: 0.8242 - val_loss: 0.5636 - val_acc: 0.6599
Epoch 5/100
 - 1s - loss: 0.2865 - acc: 0.8963 - val_loss: 0.5987 - val_acc: 0.6475
Epoch 6/100
 - 1s - loss: 0.1846 - acc: 0.9504 - val_loss: 0.6298 - val_acc: 0.6556
Epoch 7/100
 - 1s - loss: 0.1218 - acc: 0.9726 - val_loss: 0.6825 - val_acc: 0.6499
Epoch 8/100
 - 1s - loss: 0.0783 - acc: 0.9841 - val_loss: 0.7376 - val_acc: 0.6584
Epoch 9/100
 - 1s - loss: 0.0602 - acc: 0.9877 - val_loss: 0.8002 - val_acc: 0.6535
Epoch 10/100
 - 1s - loss: 0.0480 - acc: 0.9889 - val_loss: 0.8072 - val_acc: 0.6523
Epoch 11/100
 - 1s - loss: 0.0374 - acc: 0.9910 - val_loss: 0.8845 - val_acc: 0.6526
Epoch 12/100
 - 1s - loss:

Epoch 97/100
 - 1s - loss: 0.0238 - acc: 0.9922 - val_loss: 1.4616 - val_acc: 0.6541
Epoch 98/100
 - 1s - loss: 0.0284 - acc: 0.9925 - val_loss: 1.5196 - val_acc: 0.6550
Epoch 99/100
 - 1s - loss: 0.0283 - acc: 0.9922 - val_loss: 1.5235 - val_acc: 0.6514
Epoch 100/100
 - 1s - loss: 0.0316 - acc: 0.9928 - val_loss: 1.4977 - val_acc: 0.6520
0.5916784203102963


In [18]:
def cnn_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=False)
    model.add(e)
    model.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=256, verbose=2)
    y_pred = [int(round(i[0])) for i in model.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [19]:
print(cnn_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 3327 samples, validate on 3328 samples
Epoch 1/100
 - 3s - loss: 0.7428 - acc: 0.5200 - val_loss: 0.6820 - val_acc: 0.5631
Epoch 2/100
 - 1s - loss: 0.6912 - acc: 0.5512 - val_loss: 0.6787 - val_acc: 0.5616
Epoch 3/100
 - 1s - loss: 0.6743 - acc: 0.5876 - val_loss: 0.6695 - val_acc: 0.5736
Epoch 4/100
 - 1s - loss: 0.6537 - acc: 0.6213 - val_loss: 0.6576 - val_acc: 0.5865
Epoch 5/100
 - 1s - loss: 0.6310 - acc: 0.6405 - val_loss: 0.6466 - val_acc: 0.6238
Epoch 6/100
 - 1s - loss: 0.6106 - acc: 0.6733 - val_loss: 0.6424 - val_acc: 0.5992
Epoch 7/100
 - 1s - loss: 0.5798 - acc: 0.7084 - val_loss: 0.6288 - val_acc: 0.6406
Epoch 8/100
 - 1s - loss: 0.5444 - acc: 0.7547 - val_loss: 0.6066 - val_acc: 0.6484
Epoch 9/100
 - 1s - loss: 0.5041 - acc: 0.7677 - val_loss: 0.6018 - val_acc: 0.6547
Epoch 10/100
 - 1s - loss: 0.4547 - acc: 0.8115 - val_loss: 0.5973 - val_acc: 0.6544
Epoch 11/100
 - 1s - loss: 0.4169

Epoch 94/100
 - 1s - loss: 0.0184 - acc: 0.9907 - val_loss: 1.2667 - val_acc: 0.6578
Epoch 95/100
 - 1s - loss: 0.0258 - acc: 0.9919 - val_loss: 1.3183 - val_acc: 0.6481
Epoch 96/100
 - 1s - loss: 0.0237 - acc: 0.9919 - val_loss: 1.2600 - val_acc: 0.6674
Epoch 97/100
 - 1s - loss: 0.0284 - acc: 0.9904 - val_loss: 1.3308 - val_acc: 0.6686
Epoch 98/100
 - 1s - loss: 0.0249 - acc: 0.9901 - val_loss: 1.2194 - val_acc: 0.6662
Epoch 99/100
 - 1s - loss: 0.0234 - acc: 0.9922 - val_loss: 1.2583 - val_acc: 0.6707
Epoch 100/100
 - 1s - loss: 0.0239 - acc: 0.9886 - val_loss: 1.2524 - val_acc: 0.6686
0.5946343256155825


In [20]:
def cnn_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=True)
    model.add(e)
    model.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=256, verbose=2)
    y_pred = [int(round(i[0])) for i in model.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [21]:
print(cnn_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Train on 3327 samples, validate on 3328 samples
Epoch 1/100
 - 2s - loss: 0.7430 - acc: 0.5293 - val_loss: 0.6754 - val_acc: 0.5850
Epoch 2/100
 - 1s - loss: 0.6838 - acc: 0.5678 - val_loss: 0.6662 - val_acc: 0.6214
Epoch 3/100
 - 1s - loss: 0.6460 - acc: 0.6291 - val_loss: 0.6448 - val_acc: 0.6418
Epoch 4/100
 - 1s - loss: 0.5949 - acc: 0.6976 - val_loss: 0.6281 - val_acc: 0.6277
Epoch 5/100
 - 1s - loss: 0.5407 - acc: 0.7583 - val_loss: 0.6028 - val_acc: 0.6562
Epoch 6/100
 - 1s - loss: 0.4662 - acc: 0.8344 - val_loss: 0.5836 - val_acc: 0.6566
Epoch 7/100
 - 1s - loss: 0.3887 - acc: 0.8671 - val_loss: 0.5893 - val_acc: 0.6370
Epoch 8/100
 - 1s - loss: 0.3099 - acc: 0.9131 - val_loss: 0.5842 - val_acc: 0.6466
Epoch 9/100
 - 1s - loss: 0.2329 - acc: 0.9495 - val_loss: 0.5804 - val_acc: 0.6629
Epoch 10/100
 - 1s - loss: 0.1718 - acc: 0.9726 - val_loss: 0.6060 - val_acc: 0.6605
Epoch 11/100
 - 1s - loss: 0.1253 - acc: 0.9811 - val_loss: 0.6201 - val_acc: 0.6629
Epoch 12/100
 - 1s - loss:

Epoch 97/100
 - 1s - loss: 0.0168 - acc: 0.9907 - val_loss: 1.0135 - val_acc: 0.6572
Epoch 98/100
 - 1s - loss: 0.0150 - acc: 0.9934 - val_loss: 1.0207 - val_acc: 0.6529
Epoch 99/100
 - 1s - loss: 0.0144 - acc: 0.9928 - val_loss: 1.0186 - val_acc: 0.6514
Epoch 100/100
 - 1s - loss: 0.0149 - acc: 0.9925 - val_loss: 1.0548 - val_acc: 0.6484
0.6323067253299811


In [22]:
def lstm_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=False)
    model.add(e)
    model.add(LSTM(100))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=256, verbose=2)
    y_pred = [int(round(i[0])) for i in model.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [23]:
print(lstm_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Train on 3327 samples, validate on 3328 samples
Epoch 1/10
 - 113s - loss: 0.6902 - acc: 0.5497 - val_loss: 0.6825 - val_acc: 0.5607
Epoch 2/10
 - 114s - loss: 0.6801 - acc: 0.5627 - val_loss: 0.6788 - val_acc: 0.5772
Epoch 3/10
 - 117s - loss: 0.6731 - acc: 0.5816 - val_loss: 0.6752 - val_acc: 0.5895
Epoch 4/10
 - 117s - loss: 0.6645 - acc: 0.5999 - val_loss: 0.6714 - val_acc: 0.5962
Epoch 5/10
 - 115s - loss: 0.6521 - acc: 0.6141 - val_loss: 0.6719 - val_acc: 0.5799
Epoch 6/10
 - 114s - loss: 0.6396 - acc: 0.6402 - val_loss: 0.6633 - val_acc: 0.5871
Epoch 7/10
 - 117s - loss: 0.6254 - acc: 0.6507 - val_loss: 0.6547 - val_acc: 0.5980
Epoch 8/10
 - 117s - loss: 0.5979 - acc: 0.6682 - val_loss: 0.6554 - val_acc: 0.6034
Epoch 9/10
 - 117s - loss: 0.5702 - acc: 0.6985 - val_loss: 0.6645 - val_acc: 0.5974
Epoch 10/10
 - 117s - loss: 0.5515 - acc: 0.7106 - val_loss: 0.6601 - val_acc: 0.6121
0.5765824860610036


In [24]:
def lstm_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=True)
    model.add(e)
    model.add(LSTM(100))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=256, verbose=2)
    y_pred = [int(round(i[0])) for i in model.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [25]:
print(lstm_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Train on 3327 samples, validate on 3328 samples
Epoch 1/10
 - 125s - loss: 0.6872 - acc: 0.5549 - val_loss: 0.6801 - val_acc: 0.5637
Epoch 2/10
 - 125s - loss: 0.6749 - acc: 0.5705 - val_loss: 0.6747 - val_acc: 0.5649
Epoch 3/10
 - 127s - loss: 0.6616 - acc: 0.6005 - val_loss: 0.6676 - val_acc: 0.5874
Epoch 4/10
 - 123s - loss: 0.6405 - acc: 0.6369 - val_loss: 0.6669 - val_acc: 0.5844
Epoch 5/10
 - 126s - loss: 0.6082 - acc: 0.6814 - val_loss: 0.6502 - val_acc: 0.6115
Epoch 6/10
 - 125s - loss: 0.5614 - acc: 0.7096 - val_loss: 0.6503 - val_acc: 0.6223
Epoch 7/10
 - 126s - loss: 0.5185 - acc: 0.7517 - val_loss: 0.6403 - val_acc: 0.6319
Epoch 8/10
 - 124s - loss: 0.4570 - acc: 0.7902 - val_loss: 0.6832 - val_acc: 0.6337
Epoch 9/10
 - 124s - loss: 0.4016 - acc: 0.8194 - val_loss: 0.6821 - val_acc: 0.6316
Epoch 10/10
 - 123s - loss: 0.3627 - acc: 0.8455 - val_loss: 0.7437 - val_acc: 0.6439
0.5745062836624776
